In [ ]:
import tensorflow as tf
from keras.layers import Dense, LSTM, Conv1D, BatchNormalization, Flatten, Dropout, Activation, Bidirectional, MaxPooling1D, GlobalAveragePooling1D, Add
from keras.models import Model, Sequential
import numpy as np
import pandas as pd
import os

# Custom Model

In [ ]:
model = Sequential()
model.add(Conv1D(64, (4), padding= 'same', input_shape = x_tr[0].shape))
model.add(Dropout(0.2))


model.add(Conv1D(128, (4), padding='same', activation = 'relu'))
model.add(Dropout(0.2))
model.add(Conv1D(256, (4), activation= 'relu'))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(128, return_sequences= True)))
model.add(Bidirectional(LSTM(64, return_sequences= True)))

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dense(6, activation='softmax'))

# VGG16

In [ ]:
model1 = Sequential()
model1.add(Conv1D(64, (3), padding= 'same', input_shape = x_tr[0].shape, activation = 'elu'))
model1.add(Conv1D(64, (3), padding= 'same', activation = 'elu'))
model1.add(MaxPooling1D(pool_size=4, strides=2))
model1.add(Conv1D(128, (3), padding = 'same', activation = 'elu'))
model1.add(Conv1D(128, (3), padding = 'same', activation = 'elu'))
model1.add(MaxPooling1D(pool_size=4, strides=2))
model1.add(Conv1D(256, (3), padding = 'same', activation = 'elu'))
model1.add(Conv1D(256, (3), padding = 'same', activation = 'elu'))
model1.add(Conv1D(256, (3), padding = 'same', activation = 'elu'))
model1.add(MaxPooling1D(pool_size=4, strides=2))
model1.add(Conv1D(512, (3), padding = 'same', activation = 'elu'))
model1.add(Conv1D(512, (3), padding = 'same', activation = 'elu'))
model1.add(Conv1D(512, (3), padding = 'same', activation = 'elu'))
model1.add(MaxPooling1D(pool_size=4, strides=2))
model1.add(Flatten())
model1.add(Dense(256, activation='elu'))
model1.add(Dense(128, activation='elu'))
model1.add(Dense(6, activation='softmax'))

# Resnet

In [ ]:
class ResnetBlock(Model):
    """
    A standard resnet block.
    """

    def __init__(self, channels: int, down_sample=False):
        """
        channels: same as number of convolution kernels
        """
        super().__init__()

        self.__channels = channels
        self.__down_sample = down_sample
        self.__strides = [2, 1] if down_sample else [1, 1]

        KERNEL_SIZE = (3)
        
        INIT_SCHEME = "he_normal"

        self.conv_1 = Conv1D(self.__channels, strides=self.__strides[0],
                             kernel_size=KERNEL_SIZE, padding="same", activation= 'elu')
        self.bn_1 = BatchNormalization()
        self.conv_2 = Conv1D(self.__channels, strides=self.__strides[1],
                             kernel_size=KERNEL_SIZE, padding="same", activation= 'elu')
        self.bn_2 = BatchNormalization()
        self.merge = Add()

        if self.__down_sample:
            
            self.res_conv = Conv1D(
                self.__channels, strides=2, kernel_size=(1), kernel_initializer=INIT_SCHEME, padding="same")
            self.res_bn = BatchNormalization()

    def call(self, inputs):
        res = inputs

        x = self.conv_1(inputs)
        x = self.bn_1(x)
        x = tf.nn.relu(x)
        x = self.conv_2(x)
        x = self.bn_2(x)

        if self.__down_sample:
            res = self.res_conv(res)
            res = self.res_bn(res)

        # if not perform down sample, then add a shortcut directly
        x = self.merge([x, res])
        out = tf.nn.relu(x)
        return out


class ResNet18(Model):

    def __init__(self, num_classes, **kwargs):
        """
            num_classes: number of classes in specific classification task.
        """
        super().__init__(**kwargs)
        self.conv_1 = Conv1D(64, (7), strides=2,
                             padding="same", kernel_initializer="he_normal", input_shape = x_tr[0].shape)
        self.init_bn = BatchNormalization()
        self.pool_2 = MaxPooling1D(pool_size=(2), strides=2, padding="same")
        self.res_1_1 = ResnetBlock(64)
        self.res_1_2 = ResnetBlock(64)
        self.res_2_1 = ResnetBlock(128, down_sample=True)
        self.res_2_2 = ResnetBlock(128)
        self.res_3_1 = ResnetBlock(256, down_sample=True)
        self.res_3_2 = ResnetBlock(256)
        self.res_4_1 = ResnetBlock(512, down_sample=True)
        self.res_4_2 = ResnetBlock(512)
        self.avg_pool = GlobalAveragePooling1D()
        self.flat = Flatten()
        self.fc = Dense(num_classes, activation="softmax")

    def call(self, inputs):
        out = self.conv_1(inputs)
        out = self.init_bn(out)
        out = tf.nn.relu(out)
        out = self.pool_2(out)
        for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2, self.res_3_1, self.res_3_2, self.res_4_1, self.res_4_2]:
            out = res_block(out)
        out = self.avg_pool(out)
        out = self.flat(out)
        out = self.fc(out)
        return out

In [ ]:
model2 = ResNet18(6)

In [ ]:
model.compile(
            loss = 'sparse_categorical_crossentropy',
            metrics= ['accuracy'],
            optimizer= tf.keras.optimizers.Adamax(learning_rate=0.0001)
)

In [ ]:
filepath = r'.......'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint("WISDM_custom.h5", 
                                            monitor = 'val_loss',
                                            save_best_only = True,
                                            mode = 'auto',
                                            save_freq = 'epoch'
)

In [ ]:
his = model.fit(
        x_tr, y_tr,
        epochs= 100,
       validation_data= (x_test, y_test),
       callbacks= [model_checkpoint]
)